In [ ]:
import pandas as pd
pd.set_option('float.format','{:f}'.format)

import numpy as np
import matplotlib as mpl
pd.set_option('display.max_rows',999)


import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

from sklearn import metrics

import statsmodels.api as sm

import math 
from scipy import stats


**Visualization Function**

In [ ]:
legpos = 'center left'
size = 'medium'
loc=(1,0.5)
%matplotlib inline

sns.set()

def visualization(df, x, y, figsize=(12,3), hue=None, scatter=False, dist=False, cust_col='Set2', title='' ,xlabel='', ylabel='', rotation_angel=90):

    fig = plt.figure(figsize=figsize)
    if (scatter):
        ax = sns.scatterplot(x=x,y=y,data=df,hue=hue,palette=cust_col)

        plt.title(title)
        plt.xticks(rotation=rotation_angel)
        ax.set(xlabel=xlabel, ylabel=ylabel)
#     elif (dist):
#         ax = sns.displot(x=x,y=y,data=df,hue=hue,palette=cust_col)

#         plt.title(title)
#         plt.xticks(rotation=rotation_angel)
#         ax.set(xlabel=xlabel, ylabel=ylabel)

    elif (hue != None):
        ax = sns.lineplot(x=x,y=y,data=df,hue=hue,palette=cust_col)

        plt.title(title)
        plt.xticks(rotation=rotation_angel)
        plt.legend(loc=legpos,bbox_to_anchor=loc,fontsize=size)
        ax.set(xlabel=xlabel, ylabel=ylabel)
    else:
        ax = sns.lineplot(x=x,y=y,data=df,palette=cust_col)

        plt.title(title)
        plt.xticks(rotation=rotation_angel)
        ax.set(xlabel=xlabel, ylabel=ylabel)

    plt.show()
    
def visualization_3d(df, x, y, z,figsize=(8,8), title='' ,xlabel='', ylabel='', zlabel=''):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection='3d')
    ax.scatter3D(df[x], df[y], df[z], cmap='Greens')
    plt.title(title)
    ax.set(xlabel=xlabel, ylabel=ylabel,zlabel=zlabel)
    plt.show()
    
    

**Feature Engineering**

In [ ]:
def classify_data(df2,df3,df4):
    tmp2 = df2.copy()
    tmp3 = df3.copy()
    tmp4 = df4.copy()
    
    tmp2.loc[:,'type'] = 1
    tmp3.loc[:,'type'] = 2
    tmp4.loc[:,'type'] = 3
    

    tmp = pd.concat([tmp2,tmp3,tmp4])

    tmp = tmp[['X1', 'X2', 'X3','type']].reset_index(drop=True)

    # tmp = tmp.copy()
    tmp.loc[:,'X1X2'] = tmp.X1 * tmp.X2
    tmp.loc[:,'X1X3'] = tmp.X1 * tmp.X3
    tmp.loc[:,'X2X3'] = tmp.X2 * tmp.X3
    
    return tmp

def classification_accuracy(df):
    df = df.copy()
    df.loc[:,'predict_type'] = 1
    df.loc[(df['X1X2']<0) & (df['X1X3']>0),'predict_type'] = 2
    accuracy = (df['type'] == df['predict_type']).sum() / df.shape[0]
    
    return accuracy 


In [ ]:
def feature_cleaning(df,start_date,end_date,dates_exclude=['2022-08-05']):
    df = df.copy()
    
    ##### data type 
    df.index = pd.to_datetime(df.index)
    df.columns = df.columns.astype(str)
    ##### available dates 
    df = df.loc[(df.index >= start_date) & (df.index <= end_date) & (~df.index.isin(dates_exclude))]
    
    return df 

def display_features(df,feature='cap',scatter=False):
    display(feature + ' start date: '+str(df.index.min()))
    display(feature + ' end date: '+str(df.index.max()))
    tmp = df.notnull().sum(axis=1).to_frame().rename(columns={0:'cnt'})
    if not scatter:
        visualization(df=None, x=tmp.index,y=tmp.cnt,title='Number of daily available securities - '+feature)
    else:
        visualization(df=None, x=tmp.index,y=tmp.cnt,title='Number of daily available securities - '+feature, scatter=True)
        
def whether_in_universe(df):
    isin_univ = df.copy()
    isin_univ = isin_univ.fillna(0)
    isin_univ[isin_univ != 0] = 1.0
    isin_univ = isin_univ.astype('int')
    return isin_univ


In [ ]:

def get_ttm_gross_sales(financial, current_date, symbol, lag_quarter):   
    query_quarters = 4 + lag_quarter
    
    financial_symbol = financial[financial['symbol']==symbol]
    financial_segment = financial_symbol[financial_symbol['Date']<current_date].sort_values(by='QuarterNum', ascending=True).set_index('QuarterNum')
    if financial_segment.shape[0] == 0:
        return np.nan

    end_quarter_num = financial_segment.index[-1] - lag_quarter
    start_quarter_num = end_quarter_num - 3

    finalcial_cal_period = financial_segment.loc[start_quarter_num:end_quarter_num]
    if finalcial_cal_period.shape[0] > 0:
        sales_ttm = finalcial_cal_period['GrossSales'].mean() * 4
    else:
        sales_ttm = np.nan
    return sales_ttm    
    

def get_ttm_net_income(financial, current_date, symbol, lag_quarter):
    query_quarters = 4 + lag_quarter
    
    financial_symbol = financial[financial['symbol']==symbol]
    financial_segment = financial_symbol[financial_symbol['Date']<current_date].sort_values(by='QuarterNum', ascending=True).set_index('QuarterNum')
    if financial_segment.shape[0] == 0:
        return np.nan
    
    # period for filling missing values
    end_quarter_num = financial_segment.index[-1]
    start_quarter_num = end_quarter_num - query_quarters

    finalcial_cal_period = financial_segment.loc[start_quarter_num:end_quarter_num]

    # fill missing
    net_income_table = pd.DataFrame(index=range(start_quarter_num, end_quarter_num+1), columns=['FiscalQuarter', 'NetIncome'])
    net_income_table.loc[finalcial_cal_period.index, 'FiscalQuarter'] = finalcial_cal_period['FiscalQuarter']
    net_income_table.loc[finalcial_cal_period.index, 'NetIncome'] = finalcial_cal_period['NetIncome']
    net_income_table['NI_per_quarter'] = (net_income_table['NetIncome'] / net_income_table['FiscalQuarter']).fillna(method='ffill').fillna(method='bfill')
    net_income_table['NI_noncumulative'] = net_income_table['NetIncome'].diff()
    net_income_table.loc[net_income_table['FiscalQuarter']==1, 'NI_noncumulative'] = net_income_table.loc[net_income_table['FiscalQuarter']==1, 'NetIncome']
    net_income_table.loc[pd.isnull(net_income_table['NI_noncumulative']), 'NI_noncumulative'] = net_income_table.loc[pd.isnull(net_income_table['NI_noncumulative']), 'NI_per_quarter']

    # period for calculation ttm
    end_quarter_num = financial_segment.index[-1] - lag_quarter
    start_quarter_num = end_quarter_num - 3

    ni_ttm = net_income_table.loc[start_quarter_num:end_quarter_num]['NI_noncumulative'].sum()
    return ni_ttm

def fiscal_report_dates(financial,symbol):
    return financial.loc[financial['symbol']==symbol, 'Date'].values

def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean()
    s = r.std()
    z = (x-m)/s
    return z

In [ ]:
def fill_missing_values(df,ori_col = ['cp', 'op', 'adv', 'cap', 'holding_indicator',
       'technical_indicator', 'net_income_growth', 'profit_margin', 'pe_ratio',
       'ps_ratio']):
    
    ##### get industry avg
    industry_avg = df.reset_index().groupby(['date','sector']).mean()
    ### rename column 
    industry_avg = industry_avg[ori_col].reset_index()
    for col in ori_col:
        industry_avg = industry_avg.rename(columns={col:col+'_sector'})
        
    ##### merge industry avg 
    df = df.reset_index().merge(industry_avg, how='left',left_on=['date','sector'],right_on=['date','sector'])
    
    ##### fill missing values 
    for col in ori_col:
        df.loc[df[col].isnull(),col]=df.loc[df[col].isnull(),col+'_sector']
    
    df = df[['date','variable','sector']+ori_col].set_index(['date','variable']).sort_index()
    return df


**Model**

In [ ]:
def raw_features_df():
    sector_ind = sector.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'sector'})

    cp_tmp = cp.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'cp'})
    op_tmp = op.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'op'})
    adv_tmp = adv.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'adv'})
    cap_tmp = cap.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'cap'})

    holding_indicator_tmp = holding_indicator.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'holding_indicator'})
    technical_indicator_tmp = technical_indicator.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'technical_indicator'})

    net_income_growth_tmp = net_income_growth.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'net_income_growth'})
    profit_margin_tmp = profit_margin.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'profit_margin'})
    pe_ratio_tmp = pe_ratio.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'pe_ratio'})
    ps_ratio_tmp = ps_ratio.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).rename(columns={'value':'ps_ratio'})

    
    raw_features = sector_ind.merge(cp_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(op_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(adv_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(cap_tmp,left_index=True, right_index=True, how='left')

    raw_features = raw_features.merge(holding_indicator_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(technical_indicator_tmp,left_index=True, right_index=True, how='left')

    raw_features = raw_features.merge(net_income_growth_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(profit_margin_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(pe_ratio_tmp,left_index=True, right_index=True, how='left')
    raw_features = raw_features.merge(ps_ratio_tmp,left_index=True, right_index=True, how='left')
    
    return raw_features


def feature_in_univ(df, univ):
    rst = df.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable'])
    rst = rst.loc[rst.index.isin(univ.index)]
    return rst


def universe_selection(df,df2,size=10000,isfilter=False):
    df2 = df2[df2>size].copy()
    if isfilter:
        df2 = df2.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable'])
        df1 = df.copy()
        df = df.reset_index().rename(columns={'index':'date'}).melt(id_vars='date').set_index(['date','variable']).copy()
        df = df.loc[df.index.isin(df2.index)]
        df = df[['value']].reset_index().pivot(index='date',columns='variable',values='value')
        df = df[df1.columns]
    rst = pd.DataFrame(index=df.index, columns=df.columns)
    tmp = df.copy()

    for row in tmp.index:
        top_300 = tmp.loc[row,:].sort_values(ascending=False).iloc[:300].index.to_list()
        rst.loc[row,top_300] = 1
    return rst

    

In [ ]:
def get_betas(univ_brtn: pd.DataFrame):
   
    ##### get average return for each date
    avg_rtn = univ_brtn.groupby(level='date')['bd1'].mean().reset_index().rename(columns={'bd1':'avg_rtn'})
    ##### get securities included (500)
    sec_list = univ_brtn.index.unique('sector')
    ##### for each security, calculate beta
    rst = pd.DataFrame(columns=['beta'])
    for sec in sec_list:
        tmp = univ_brtn.loc[univ_brtn.index.get_level_values('sector') == sec].reset_index()
        sec_rtn = tmp['bd1'].values
        rtn = avg_rtn.merge(tmp, how='right',left_on='date',right_on='date')
        mkt_rtn = rtn['avg_rtn'].values
        rst.loc[sec,'beta'] = np.cov(sec_rtn,mkt_rtn)[0,1]/np.var(mkt_rtn)

    return rst['beta'].astype(float)

def adjust_market_impact(data,univ,dt,signal,col='momentum'):
   
    ##### adjust for market impacts - beta 
    ### get beta 
    beta = get_betas(univ_brtn=data)
    
    ##### adjust for sector impacts 
    ### create dummy variables for sector
    sector = univ.loc[(univ.index.get_level_values('date')==dt)][['sector']]
    sector = pd.get_dummies(data=sector['variable'],prefix='sector').droplevel('date')
    
    ##### perform linear regression to adjust market and sector impacts 
    X = pd.concat([beta,sector], axis=1)
    adj_signal = signal - LinearRegression().fit(X, signal).predict(X)
    
    adj_signal.name = col
    # another way to adjust mkt & sector impacts can be: perform stepwise regression
    # adjust mkt impacts first 
    # then adjust for sector impacts 
    return adj_signal


In [ ]:
def get_signals(df,features_clean):
    ######## return 
    close_rtn = df.pct_change(1)
    fwd_rtn_20 = df.pct_change(20).shift(-1)

    ##### return ts mean 
    rtn_1w = close_rtn.rolling(window=5).mean()
    rtn_1m = close_rtn.rolling(window=20).mean()
    rtn_6m = close_rtn.rolling(window=120).mean()
    rtn_1y = close_rtn.rolling(window=252).mean()
    ##### return ts zscore 
    rtn_z_1w = zscore(x=close_rtn,window=5)
    rtn_z_1m = zscore(x=close_rtn,window=20)
    rtn_z_6m = zscore(x=close_rtn,window=120)
    rtn_z_1y = zscore(x=close_rtn,window=252)
    ##### return ts skew 
    rtn_skew_1w = close_rtn.rolling(window=5).skew()
    rtn_skew_1m = close_rtn.rolling(window=20).skew()
    rtn_skew_6m = close_rtn.rolling(window=120).skew()
    rtn_skew_1y = close_rtn.rolling(window=252).skew()

    ##### return ts diff 
    # rtn_diff_1w = ts_diff(dt=close_rtn, lookback1=5,lookback2=10)
    # rtn_diff_1m = ts_diff(dt=close_rtn, lookback1=20,lookback2=40)
    # rtn_diff_6m = ts_diff(dt=close_rtn, lookback1=120,lookback2=240)

    ##### up returns 
    up_rtn = close_rtn.copy()
    up_rtn[up_rtn<0] = 0
    ##### up return ts mean 
    up_rtn_1w = up_rtn.rolling(window=5).mean()
    up_rtn_1m = up_rtn.rolling(window=20).mean()
    up_rtn_6m = up_rtn.rolling(window=120).mean()
    up_rtn_1y = up_rtn.rolling(window=252).mean()

    ##### down returns 
    down_rtn = close_rtn.copy()
    down_rtn[down_rtn>0] = 0
    ##### down return ts mean 
    down_rtn_1w = down_rtn.rolling(window=5).mean()
    down_rtn_1m = down_rtn.rolling(window=20).mean()
    down_rtn_6m = down_rtn.rolling(window=120).mean()
    down_rtn_1y = down_rtn.rolling(window=252).mean()

    ##### adv ts zscore 
    adv_z_1w = zscore(x=adv_clean,window=5)
    adv_z_1m = zscore(x=adv_clean,window=20)
    adv_z_6m = zscore(x=adv_clean,window=120)
    adv_z_1y = zscore(x=adv_clean,window=252)
    ##### adv ts mean 
    adv_1w = adv_clean.rolling(window=5).mean()
    adv_1m = adv_clean.rolling(window=20).mean()
    adv_6m = adv_clean.rolling(window=120).mean()
    adv_1y = adv_clean.rolling(window=252).mean()

    ##### cap ts zscore 
    cap_z_1w = zscore(x=cap_clean,window=5)
    cap_z_1m = zscore(x=cap_clean,window=20)
    cap_z_6m = zscore(x=cap_clean,window=120)
    cap_z_1y = zscore(x=cap_clean,window=252)
    ##### adv ts mean 
    cap_1w = cap_clean.rolling(window=5).mean()
    cap_1m = cap_clean.rolling(window=20).mean()
    cap_6m = cap_clean.rolling(window=120).mean()
    cap_1y = cap_clean.rolling(window=252).mean()

    signals = pd.DataFrame({'fwdrtn':fwd_rtn_20.melt()['value'].values,
                            'net_income_growth':features_clean.net_income_growth.values,
                            'profit_margin':features_clean.profit_margin.values,
                            'pe_ratio':features_clean.pe_ratio.values,
                            'ps_ratio':features_clean.ps_ratio.values,
                            'technical_indicator':features_clean.technical_indicator.values,
                            'rtn_1w':rtn_1w.melt()['value'].values,'rtn_1m':rtn_1m.melt()['value'].values,'rtn_6m':rtn_6m.melt()['value'].values,'rtn_1y':rtn_1y.melt()['value'].values,
                            'rtn_z_1w':rtn_z_1w.melt()['value'].values,'rtn_z_1m':rtn_z_1m.melt()['value'].values,'rtn_z_6m':rtn_z_6m.melt()['value'].values,'rtn_z_1y':rtn_z_1y.melt()['value'].values,
                           'rtn_skew_1w':rtn_skew_1w.melt()['value'].values,'rtn_skew_1m':rtn_skew_1m.melt()['value'].values,'rtn_skew_6m':rtn_skew_6m.melt()['value'].values,'rtn_skew_1y':rtn_skew_1y.melt()['value'].values,
                           'up_rtn_1w':up_rtn_1w.melt()['value'].values,'up_rtn_1m':up_rtn_1m.melt()['value'].values,'up_rtn_6m':up_rtn_6m.melt()['value'].values,'up_rtn_1y':up_rtn_1y.melt()['value'].values,
                           'down_rtn_1w':down_rtn_1w.melt()['value'].values,'down_rtn_1m':down_rtn_1m.melt()['value'].values,'down_rtn_6m':down_rtn_6m.melt()['value'].values,'down_rtn_1y':down_rtn_1y.melt()['value'].values,
                            'adv_z_1w':adv_z_1w.melt()['value'].values,'adv_z_1m':adv_z_1m.melt()['value'].values,'adv_z_6m':adv_z_6m.melt()['value'].values,'adv_z_1y':adv_z_1y.melt()['value'].values,
                            'adv_1w':adv_1w.melt()['value'].values,'adv_1m':adv_1m.melt()['value'].values,'adv_6m':adv_6m.melt()['value'].values,'adv_1y':adv_1y.melt()['value'].values,
                            'cap_z_1w':cap_z_1w.melt()['value'].values,'cap_z_1m':cap_z_1m.melt()['value'].values,'cap_z_6m':cap_z_6m.melt()['value'].values,'cap_z_1y':cap_z_1y.melt()['value'].values,

                           })
    signals.index=features_clean.index
    return signals


def clean_backtestinput_format(df,start_date,end_date,isfwd=False):
    if isfwd:
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df[df.abs() > 1] = np.nan
    df = df.sort_index().loc[start_date:end_date]
    return df

def get_alpha(df,signals,pos_thres=0.0005, neg_thres=-0.0005, col='fwdrtn',isfilter=False):
    import warnings
    from scipy.linalg import LinAlgWarning
    warnings.filterwarnings(action='ignore', category=LinAlgWarning, module='sklearn')
    
    df = df.copy()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    correlation = df.corr()
    if isfilter:
        features = correlation.loc[(correlation.fwdrtn > pos_thres) | (correlation.fwdrtn<neg_thres)][[col]].sort_values(col).index.to_list()
    else:
        features = correlation.index.to_list()
    features.remove(col)
    
    train = df.dropna(subset=[col]+features)
    X = train[features]
    Y = train[col]
    model = Ridge(alpha=0.51)
    Ridgemodel = model.fit(X,Y)
    
    available = df.dropna(subset=features).copy()
    
    rst = pd.DataFrame(data=Ridgemodel.predict(available[features]),index=available.index,columns=['alpha'])
    alpha = pd.DataFrame(index=signals.index,columns=['alpha'])
    tmp = alpha.merge(rst,left_index=True,right_index=True,how='left')
    alpha = tmp[['alpha_y']].reset_index().pivot(index='date',columns='variable',values='alpha_y')
    
    return alpha 



    
    


In [ ]:
def pred_model(df):
    df = df.copy()
    
    df.Y = -df.X3
    
    df.loc[abs(df.X1) < 0.1,'Y'] = 0.1  
    df.loc[(abs(df.X1) >= 0.1) & (df.X1*df.X2 < 0) & (df.X1*df.X3 > 0), 'Y'] = df.loc[(abs(df.X1) >= 0.1) & (df.X1*df.X2 < 0) & (df.X1*df.X3 > 0), 'X1'] + df.loc[(abs(df.X1) >= 0.1) & (df.X1*df.X2 < 0) & (df.X1*df.X3 > 0), 'X2'] + df.loc[(abs(df.X1) >= 0.1) & (df.X1*df.X2 < 0) & (df.X1*df.X3 > 0), 'X3']
    
    
    return df 
           

**Backtesting**

In [2]:
def Correlation(s1, s2, method='pearson'):
    corr = None
    not_nan_loc = (~np.isnan(s1)) & (~np.isnan(s2))


    if not_nan_loc.sum() < 2:
        return np.nan

    s1 = s1[not_nan_loc]
    s2 = s2[not_nan_loc]
    
    if method == 'pearson':
        corr = stats.pearsonr(s1, s2)[0]
    elif method == 'spearman':
        corr = stats.spearmanr(s1, s2)[0]
    return corr

def Regression(x, y):
    if x.shape[0] < 3:
        return np.nan, np.nan, np.nan
    
    x = x.reshape(-1, 1)
    n, k = x.shape[0], 1
    reg = LinearRegression().fit(x, y)
    y_hat = reg.predict(x)
    residual = y - y_hat
    coef = reg.coef_
    intercept = reg.intercept_

    sigma_hat = sum(residual ** 2) / (n - k - 1)  # estimate of error term variance
    variance_beta_hat = sigma_hat * np.linalg.inv(np.matmul(x.transpose(), x))
    t_stat = coef / np.sqrt(variance_beta_hat.diagonal())
    return t_stat, coef, intercept


class SingleFactorAnalysis:

    def __init__(self, forward_ret, alpha_df, tradable_df, freq):

        self.alpha_df = alpha_df.copy()
        self.alpha_np = self.alpha_df.values

        self.tradable_df = tradable_df
        self.tradable_np = self.tradable_df.values

        self.alpha_df_tradable = alpha_df.copy()
        self.alpha_df_tradable[self.tradable_df == 0] = 0
        self.alpha_np_tradable = self.alpha_df_tradable.values

        self.fwd_rtn = forward_ret
        self.fwd_rtn_np = self.fwd_rtn.values

        self.fwd_rtn_norm = self.fwd_rtn.subtract(self.fwd_rtn.mean(axis=1), axis=0)
        self.fwd_rtn_norm_np = self.fwd_rtn_norm.values

        self.dates_len = self.alpha_np.shape[0]
        self.dates_index = self.alpha_df.index
        self.sname = self.alpha_df.columns

        self.points_per_year = freq


        # group
        self.group_num = 5
        self.group_ptf_rtn_np = np.zeros((self.dates_len, self.group_num))
        self.group_ptf_rtn_df = None

        # IC
        self.IC_np = np.zeros(self.dates_len)
        self.IC_series = None

        # regression
        self.tstats_np = np.zeros(self.dates_len)
        self.tstats_series = None

        self.factor_ret_np = np.zeros(self.dates_len)
        self.factor_ret_series = None

        self.factor_alpha_np = np.zeros(self.dates_len)
        self.factor_alpha_series = None

        self.performance = {}

    def Statistics(self):

        for i in range(self.dates_len-1):

            tradable_loc = self.tradable_np[i, :] == 1

            # alpha calculation
            alpha_currentdate = self.alpha_np[i, tradable_loc]

            fwd_rtn_currentdate = self.fwd_rtn_np[i, tradable_loc]
            fwd_rtn_norm_currentdate = self.fwd_rtn_norm_np[i, tradable_loc]

            # group portfolios
            tradable_num = len(alpha_currentdate)
            num_per_group = int(tradable_num/self.group_num)
            ind = np.argsort(alpha_currentdate)  # ascending
            for j in range(self.group_num):
                ind_this_group = ind[j*num_per_group:(j+1)*num_per_group]
                fwd_rtn_group = fwd_rtn_currentdate[ind_this_group]
                self.group_ptf_rtn_np[i, j] = fwd_rtn_group[~np.isnan(fwd_rtn_group)].mean()

            not_nan_loc = (~np.isnan(alpha_currentdate)) & (~np.isnan(fwd_rtn_currentdate))


            if (~not_nan_loc).all():
                # all alpha/fwd_rtn are nan, skip today
                continue

            # IC method
            self.IC_np[i] = Correlation(alpha_currentdate[not_nan_loc], fwd_rtn_currentdate[not_nan_loc], method='spearman')

            # t stats
            tstats, factor_return, factor_alpha = Regression(alpha_currentdate[not_nan_loc], fwd_rtn_currentdate[not_nan_loc])
            self.tstats_np[i] = tstats
            self.factor_ret_np[i] = factor_return
            self.factor_alpha_np[i] = factor_alpha

        # factor weights

        weight_df = self.alpha_df_tradable.div(self.alpha_df_tradable.abs().sum(axis=1), axis=0).fillna(0)

        # factor portfolio returns
        alpha_returns_df = weight_df * self.fwd_rtn
        self.ptf_returns = alpha_returns_df.sum(axis=1)

        self.group_ptf_rtn_df = pd.DataFrame(self.group_ptf_rtn_np,
                                             index=self.dates_index,
                                             columns=['group ' + str(i) for i in range(self.group_num, 0, -1)])
        self.IC_series = pd.Series(self.IC_np, index=self.dates_index).fillna(0)
        self.IC_cum_series = self.IC_series.cumsum()
        self.tstats_series = pd.Series(self.tstats_np, index=self.dates_index).fillna(0)
        self.factor_ret_series = pd.Series(self.factor_ret_np, index=self.dates_index).fillna(0)
        self.factor_alpha_series = pd.Series(self.factor_alpha_np, index=self.dates_index).fillna(0)

        self.performance['IC mean'] = self.IC_series.mean()
        self.performance['IC std'] = self.IC_series.std()
        self.performance['ICIR'] = self.IC_series.mean()/self.IC_series.std() * np.sqrt(self.points_per_year)
        self.performance['t-stats mean'] = self.tstats_series.mean()
#         self.performance['Annual Return'] = self.ptf_returns.mean() * self.points_per_year
        self.performance['Factor Portfolio Return'] = self.ptf_returns.mean() * self.points_per_year
        self.performance['Factor Portfolio Sharpe Ratio'] = self.ptf_returns.mean() / self.ptf_returns.std() * np.sqrt(self.points_per_year)

    def PlotResult(self):
        self._CumulativeAlphaReturns()
        self._GroupPortfolios()
        self._IC()
        self._AlphaDecay()
        plt.show()

        print(self.performance)


    def _CumulativeAlphaReturns(self):

        self.performance['AnnualReturn'] = self.ptf_returns.mean() * self.points_per_year

        plt.figure(figsize=(12, 3))
        plt.plot(self.ptf_returns.cumsum())
        plt.title('Factor Weighted Long/Short Portfolio Cumulative Return')

    def _GroupPortfolios(self):

        plt.figure(figsize=(12, 3))
        plt.plot(self.group_ptf_rtn_df.cumsum())
        plt.legend(self.group_ptf_rtn_df.columns)
        plt.title('Cumulative Return by Quantile')

        plt.figure(figsize=(12, 3))
        plt.plot((self.group_ptf_rtn_df.iloc[:, -1] - self.group_ptf_rtn_df.iloc[:, 0]).cumsum())
        plt.title('Top Minus Bottom Quantile Cumulative Return')

        ptf_rtn = pd.DataFrame()
        ptf_rtn['Group bottom quantile'] = self.group_ptf_rtn_df['group 5']
        ptf_rtn['Group top quantile'] = self.group_ptf_rtn_df['group 1']

        # plt.figure(figsize=(16, 9))
        # plt.plot(ptf_rtn.cumsum())
        # plt.legend(ptf_rtn.columns)
        # plt.title('')

    def _IC(self):

        plt.figure(figsize=(12, 3))
        plt.bar(x=self.IC_series.index, height=self.IC_series.values)
        plt.plot(self.IC_series.rolling(window=5).mean())
        plt.title('IC mean')

        plt.figure(figsize=(12, 3))
        plt.plot(self.IC_cum_series.values)
        plt.title('IC Cumulative Sum')

    def _AlphaDecay(self):
        alpha_decay_np = np.zeros((self.dates_len, 10))
        decay_period = range(alpha_decay_np.shape[1])

        for i in range(self.dates_len-decay_period[-1]):
            for j in decay_period:
                alpha_decay_np[i, j] = Correlation(self.alpha_np[i, :], self.alpha_np[i+j, :])

        alpha_decay_df = pd.DataFrame(alpha_decay_np,
                                      index=self.dates_index,
                                      columns=decay_period).mean(axis=0)

        plt.figure(figsize=(12, 3))
        plt.plot(alpha_decay_df)
        plt.title('Alpha Decay')

        
        